# Converting the school registry from xml to csv

## Sources

- [gov.cz: Rejstřík škol a školských zařízení - celá ČR ](https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2F00022985%2F63989c80e16fc31c77e23ab529c76b52)

In [78]:
import pandas as pd


In [79]:
fname = 'vrejcelk.xml'
df = pd.read_xml(fname, encoding='utf-8', xpath=".//PravniSubjekt", elems_only=True)
df

,RedIzo,ICO,Reditelstvi,SkolyZarizeni
0,600000206,49625918,NaN,NaN
1,600000222,61379310,NaN,NaN
2,600000231,25765710,NaN,NaN
3,600000249,60437171,NaN,NaN
4,600000257,25143701,NaN,NaN
...,...,...,...,...
9969,691018375,21781541,NaN,NaN
9970,691018391,21757194,NaN,NaN
9971,691018405,21357331,NaN,NaN
9972,691018413,2311445,NaN,NaN


In [80]:
import xml.etree.ElementTree as ET
import pandas as pd

# xml_data = open('rejstrik_skol_test.xml', 'r').read()  # Read file vrejcelk.xml
xml_data = open('vrejcelk.xml', 'r').read()
root = ET.XML(xml_data)  # Parse XML
root



<Element 'ExportDat' at 0x0000023F7031D800>

In [81]:
def parse_element_not_working(el, item:dict, tags:set, loop_inside:set):

    ret_lines = list()

    if len(list(el)) == 0:
        if el.tag in tags:
            item[el.tag] = el.text
        return None

    else:
        if True or el.tag in loop_inside:

            for child in list(el):
                new_item = {}
                lines = parse_element(child, new_item, tags, loop_inside)
                if lines:
                    for line in lines:
                        ret_lines.append(item | line)
                else:
                    ret_lines.append(item | new_item)


        else:
            for child in list(el):
                lines = parse_element(child, item, tags, loop_inside)
                if lines:
                    for line in lines:
                        ret_lines.append(line)
                else:
                    ret_lines.append(item)
    return ret_lines

In [82]:
def parse_element(el, tags:set, loop_inside:set):

    ret_lines = list()

    if len(list(el)) == 0:
        return None

    local_vals = dict()
    local_lines = list()

    for child in list(el):
        if len(list(child)) == 0:
            if child.tag in tags:
                local_vals[child.tag] = child.text
        else:
            new_lines = parse_element(child, tags, loop_inside)
            new_local_lines = list()
            if new_lines:
                for line in new_lines:
                    if local_lines:
                        for loc_line in local_lines:
                            if set(line.keys()) == set(loc_line.keys()):
                                new_local_lines.append(line)
                                new_local_lines.append(loc_line)
                            else:
                                new_local_lines.append(loc_line | line)
                    else:
                        new_local_lines.append(line)
                local_lines = new_local_lines.copy()


    if local_lines:
        for line in local_lines:
            ret_lines.append(local_vals | line)
    elif local_vals:
        ret_lines.append( local_vals)
    return ret_lines


In [ ]:
tags = {"RedIzo", "ICO",
            "RedZkracenyNazev", "RedRUAINKod", "RedAdresa1", "RedAdresa2", "RedAdresa3",
                "PravniForma", "DruhZrizovatele", "Okres", "ORP",
            "ZrizICO",
            "IZO",
            "SkolaDruhTyp",
                "IDMista",
                "MistoDruhTyp",
                "MistoRUAINKod",
                "MistoAdresa1",
                "MistoAdresa2",
                "MistoAdresa3"
}

loop_inside = {"SkolyZarizeni"}



data = []

for child in root:
    lines = parse_element(child, tags, loop_inside)
    if lines:
        for line in lines:
            data.append(line)


df = pd.DataFrame(data)  # Write in DF and transpose it
df

In [60]:
df.head(10)

,RedZkracenyNazev,RedRUAINKod,RedAdresa1,RedAdresa2,RedAdresa3,PravniForma,DruhZrizovatele,Okres,ORP,RedIzo,ICO,ZrizICO,IDMista,MistoDruhTyp,MistoRUAINKod,MistoAdresa1,MistoAdresa2,MistoAdresa3,IZO,SkolaDruhTyp
0,Mateřská škola sv. Voršily v Praze,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,641,6,CZ0111,CZ01101,600000206,49625918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mateřská škola sv. Voršily v Praze,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,641,6,CZ0111,CZ01101,600000206,49625918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mateřská škola sv. Voršily v Praze,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,641,6,CZ0111,CZ01101,600000206,49625918,00408263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600000206,49625918,NaN,049625918,Mateřská škola,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,049625918,A00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600000206,49625918,NaN,102413096,Školní jídelna,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,102413096,L11


In [61]:
df[df['RedIzo'] == '600000206']

,RedZkracenyNazev,RedRUAINKod,RedAdresa1,RedAdresa2,RedAdresa3,PravniForma,DruhZrizovatele,Okres,ORP,RedIzo,ICO,ZrizICO,IDMista,MistoDruhTyp,MistoRUAINKod,MistoAdresa1,MistoAdresa2,MistoAdresa3,IZO,SkolaDruhTyp
0,Mateřská škola sv. Voršily v Praze,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,641,6,CZ0111,CZ01101,600000206,49625918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mateřská škola sv. Voršily v Praze,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,641,6,CZ0111,CZ01101,600000206,49625918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mateřská škola sv. Voršily v Praze,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,641,6,CZ0111,CZ01101,600000206,49625918,00408263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600000206,49625918,NaN,049625918,Mateřská škola,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,049625918,A00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600000206,49625918,NaN,102413096,Školní jídelna,21702101,Ostrovní 139/11,Nové Město,110 00 Praha 1,102413096,L11
